In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from collections import defaultdict

In [2]:
win_conditions = [(0, 1, 2), (3, 4, 5), (6, 7, 8),
                        (0, 3, 6), (1, 4, 7), (2, 5, 8),
                        (0, 4, 8), (2, 4, 6)]


In [3]:

state_rewards = {}
def init_state_space():
    possible_values = [-1, 0, 1]
    total_states = list(itertools.product(possible_values, repeat=9))
    state_space = [list(state) for state in total_states]
    return state_space


all_states = init_state_space()

len(all_states)

19683

In [4]:
def invalid_state(state): #returns True if the state is invalid else return False
    
    state = list(state)
    num_minusone = 0
    num_plusone = 0

    for i in range(len(state)):
        if(state[i] == -1):
            num_minusone+=1
        if(state[i] == 1):
            num_plusone+=1

    diff = (num_minusone - num_plusone)

    if(diff <= 1 and diff >= 0):
        return False
    
    return True


def check_win_minusone(state): #returns true if -1 wins
    

    for i in win_conditions:
        if(state[i[0]] == -1 and state[i[1]] == -1 and state[i[2]] == -1):
            return True
        
    return False
    

def check_win_one(state): #returns True if +1 wins



    for i in win_conditions:
        if(state[i[0]] == 1 and state[i[1]] == 1 and state[i[2]] == 1):
            return True
        
    return False
    

def check_draw(state): #returns True if draw

    zeros = 0

    for i in state:
        if i == 0:
            zeros+=1


    if(check_win_minusone(state) == False and check_win_one(state) == False and zeros == 0):
        return True
    
    return False
  

def check_still_going(state): #returns True if match still going on
    
    if(check_win_minusone(state)):
        return False
    
    if(check_win_one(state)):
        return False
    
    if(check_draw(state)):
        return False
    
    return True


def current_player_turn(state):

    number_one = 0
    number_minusone = 0

    for i in state:
        if i == -1:
            number_minusone+=1
            
        elif i == 1:
            number_one+=1

    if(number_minusone - number_one == 0):
        return -1
    
    if(number_minusone - number_one == 1):
        return 1


def get_empty_positions(state):
    empty_pos = []


    for i in range(len(state)):
        if state[i] == 0:
            empty_pos.append(i)

    return empty_pos



def index_max_val(dict_obj):
    
    if not dict_obj:
        return None  # Return None for an empty dict_obj

    max_key = max(dict_obj, key=dict_obj.get)
    max_value = dict_obj[max_key]
    return max_key, max_value


def index_min_val(dict_obj):
    if not dict_obj:
        return None  # Return None for an empty dict_obj

    min_key = min(dict_obj, key=dict_obj.get)
    min_value = dict_obj[min_key   ]

    return min_key , min_value

In [5]:
pruned_states = []

for i in range(len(all_states)):
    if(invalid_state(all_states[i])):
        continue
    pruned_states.append(all_states[i])


len(pruned_states)

6046

In [6]:
value = [-1,1,0,-1,1,0,-1,0,0]
index_val = pruned_states.index(value)

pruned_states[index_val]
index_val

1715

In [7]:
state_configurations = np.zeros((len(pruned_states), 9))
state_rewards = np.zeros(len(pruned_states))

for i in range(len(pruned_states)):
    
    state_configurations[i] = np.array(pruned_states[i])

    if(check_win_minusone(pruned_states[i])):
        state_rewards[i] = 1
        continue
    if(check_win_one(pruned_states[i])):
        state_rewards[i] = -1
        continue

state_configurations.setflags(write=False)
state_rewards.setflags(write=False)


In [8]:
# len(state_configurations)

testing = [-1,1,0,-1,1,0,-1,0,0]
testing_state =  np.where(np.all(state_configurations == testing, axis=1))[0][0]

testing_state

state_configurations[1715]

array([-1.,  1.,  0., -1.,  1.,  0., -1.,  0.,  0.])

In [9]:
state_configurations[5324]

array([ 1.,  0.,  0., -1.,  0.,  1., -1., -1.,  0.])

In [10]:
# for i in range(len(state_rewards)):
#     print(f"State {i}: {state_configurations[i]} and rewards = {state_rewards[i]}")


In [11]:
def get_state_configuration(numpy_obj, state):
    return numpy_obj[state]

In [12]:
def value_iteration(epsilon, gamma):
    
    TOTAL_STATES = 6046
    MAX_ITER = 10
    value_table_1 = np.zeros(TOTAL_STATES)
    value_table_2 = np.zeros(TOTAL_STATES)
    optimal_policies = np.zeros(TOTAL_STATES)

    cur_iter = 0
    
    while cur_iter < MAX_ITER:

        for state in range(TOTAL_STATES):

            if(not check_still_going(state_configurations[state])):
                continue

            
            current_player = current_player_turn(state_configurations[state])


            if(current_player == -1):

                possibile_positions = get_empty_positions(state_configurations[state])

                possible_configurations = []


                for i in possibile_positions:
                    
                    
                    # temp_1 = state_configurations[state]
                    temp = [0 for i in range(9)]
                    # temp = state_configurations[state]
                    # temp = get_state_configuration(state_configurations, state)
                    temp = np.array(state_configurations[state])

                    temp[i] = - 1

                    possible_configurations.append(temp)


                possible_states = []

                for i in possible_configurations:

                    # next_state = np.where(state_configurations == i)[0][0]
                    next_state = np.where(np.all(state_configurations == np.array(i), axis=1))[0][0]


                    possible_states.append(next_state)


                q_table = np.zeros(len(possible_states))

                for i in range(len(q_table)):

                    q_table[i] = state_rewards[possible_states[i]] + gamma*value_table_1[possible_states[i]]

                
                value_table_2[state] = np.max(q_table)
                index_max = np.argmax(q_table)
                optimal_policies[state] = possible_states[index_max]
            
            
            elif(current_player == 1):

                possibile_positions = get_empty_positions(state_configurations[state])

                possible_configurations = []


                for i in possibile_positions:

                    
                    # temp = np.array(state_configurations[state])
                    # temp = [0 for i in range(9)]
                    # temp = state_configurations[state]
                    # temp = get_state_configuration(state_configurations,state)
                    temp = np.array(state_configurations[state])
                    temp[i] = 1

                    possible_configurations.append(temp)

                possible_states = []

                for i in possible_configurations:

                    # next_state = np.where(state_configurations == i)[0][0]
                    next_state = np.where(np.all(state_configurations == np.array(i), axis=1))[0][0]

                    possible_states.append(next_state)

                q_table = np.zeros(len(possible_states))

                for i in range(len(q_table)):

                    q_table[i] = state_rewards[possible_states[i]] + gamma*value_table_1[possible_states[i]]

                
                value_table_2[state] = np.min(q_table)
                index_max = np.argmin(q_table)
                optimal_policies[state] = possible_states[index_max]

        

        delta = np.max(value_table_2) - np.max(value_table_1)

        print(delta)


        if(abs(delta) > 0 and abs(delta) < epsilon):
            break
        
        # print(delta)

        value_table_1 = value_table_2.copy()

        value_table_2 = np.zeros(TOTAL_STATES)

        cur_iter += 1

    return optimal_policies, value_table_1




            


In [13]:
optimal_policies, optimal_values = value_iteration(0.0000000001,0.95)

1.0
0.0
0.0
0.0


KeyboardInterrupt: 

In [ ]:
# print(optimal_policies[5324])

In [ ]:
state_configurations[5324]

In [ ]:
# array([ 1.,  1.,  1., -1.,  1.,  1., -1., -1.,  1.])

In [ ]:
state_configurations[1715]

In [ ]:
testing = [-1,-1,0,-1,0,0,1,1,0]

cur_state = np.where(np.all(state_configurations == testing, axis=1))[0][0]

state_configurations[cur_state]


In [ ]:
target_state = optimal_policies[cur_state]

In [ ]:
state_configurations[int(target_state)]

In [ ]:
import pickle

# open a file, where you ant to store the data
file = open('state_action', 'wb')

# dump information to that file
pickle.dump(optimal_policies, file)

# close the file
file.close()

In [ ]:
# open a file, where you ant to store the data
file = open('state_configuration', 'wb')

# dump information to that file
pickle.dump(state_configurations, file)

# close the file
file.close()